## Setup

In [ ]:
from src.data.setup import download_and_extract

download_and_extract(
    url="https://drive.google.com/uc?id=1VAc_MFUAlrtqZanxwYRRtGYVLFunKABf",
    zip_path="data.zip",
    extract_dir="data",
    verbose=True,
    remove_zip=True
)

## Train DT

In [ ]:
from src.models.decision_tree.train import main as train_decision_tree
from src.models.hoeffding.train import main as train_hoeffding_tree
from src.models.weighted_forest.train import main as train_weighted_forest

# train_decision_tree()
# train_hoeffding_tree()
train_weighted_forest()

## Launch game

In [ ]:
from src.main import main

mode = "ct" # choices: "human", "pc", "dt", "ht", "ct"
main(mode)